**OBJECTIVE:** Visualise data collected for further analysis and summarisation.

**AUTHOR:** Joshua Xu

**LAST EDITED:** 2024-10-28

---


# Data Visualisation 

In this notebook, I will be visualising the data I have collected in a more sophisticated way to support my [analysis](./DataAnalysis/DataAnalysis.ipynb)

Let's start off by importing both the data we need and the modules we require:

In [1]:
import json
import requests
import pandas as pd
import numpy as np
from lets_plot import *
LetsPlot.setup_html()

Running the saved [.py file](df_weather_code.py), I will be working with the same data frame as before: 

In [2]:
%run df_weather_code.py